# Ray Tune - Basic Tune Concepts and Steps: Warming Up with Ray Tune Hyperparameter Tuning

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

This lesson introduces Ray tune's key concepts using a trivial examples. This example is derived from [Ray Tune basic example](https://docs.ray.io/en/latest/tune/examples/tune_basic_example.html). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 ![](https://docs.ray.io/en/latest/_images/tune-workflow.png)


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


### Install Ray Tune
[Ray Tune](https://docs.ray.io/en/master/installation.html#official-releases) requires a separate install. 

### 1. Setup training using Trainable APIs

In [1]:
import time

import ray
from ray import tune

Let's define our objective function

In [2]:
def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1

Next, we define a Trainable used by Tune using Tune's [Functional API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#function-api)

In [3]:
def easy_objective(config):
    # fetch our Hyperparameters sent as arguments
    width, height = config["width"], config["height"]
    # Iterate over number of steps
    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        # Here our objective function is the evaluation_fn
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)

In [4]:
ray.init(ignore_reinit_error=True)

2022-03-16 16:00:12,697	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-16_16-00-10_301470_58630/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-16_16-00-10_301470_58630/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2022-03-16_16-00-10_301470_58630',
 'metrics_export_port': 57604,
 'gcs_address': '127.0.0.1:61096',
 'address': '127.0.0.1:61096',
 'node_id': '01128a70848b91464878eacd94cc255c7b02ca4ca8048cafb1580137'}

### Step 2. Use tune API to execute tuning

This will do a grid search over the `activation` parameter. This means
that each of the two values (`relu` and `tanh`) will be sampled once
for each sample (`num_samples`). We end up with 2 * N = 2N samples, where is `num_samples`
The `width` and `height` parameters are sampled randomly.
`steps` is a constant parameter.

The `tune.run(..)` API returns a large [analysis](https://docs.ray.io/en/latest/tune/api_docs/analysis.html#analysis-tune-analysis) object. 

In [5]:
analysis = tune.run(
    easy_objective,
    metric="mean_loss",
    mode="min",
    num_samples=5,
    # Define our hypyerparameter search space
    config={
        "steps": 5,
        "width": tune.uniform(0, 20),
        "height": tune.uniform(-100, 100),
        "activation": tune.grid_search(["relu", "tanh"]),
    },
    verbose=1
)

2022-03-16 16:00:23,455	INFO tune.py:639 -- Total run time: 2.53 seconds (1.84 seconds for the tuning loop).


### Step 3. Analyse the results

In [6]:
print("Best hyperparameters found were: ", analysis.best_config)

Best hyperparameters found were:  {'steps': 5, 'width': 6.587070281446186, 'height': -26.885495076985137, 'activation': 'relu'}


Alternatively, you can examine a Pandas dataframe

In [7]:
analysis.results_df.head(5)

/usr/local/anaconda3/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:281: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,iterations,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,date,...,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.steps,config.width,config.height,config.activation
trial_id,,,,,,,,,,,,,,,,,,,,,
daef1_00000,4,10.403145,0.102530,True,None,None,5,-10.403145,f045f83e5eb94e50bb47b023f4e1ac90,2022-03-16_16-00-23,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.528258,0,5,"0_activation=relu,height=84.481,width=10.287",5,10.287164,84.480597,relu
daef1_00001,4,6.399146,0.128668,True,None,None,5,-6.399146,6589dda1beb94cd59fcb3baaf0ad6fa6,2022-03-16_16-00-23,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.551824,0,5,"1_activation=tanh,height=32.545,width=5.45",5,5.450001,32.544925,tanh
daef1_00002,4,4.781625,0.105659,True,None,None,5,-4.781625,5b214c0fc1854efe9300e12eef47e3e0,2022-03-16_16-00-23,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.520012,0,5,"2_activation=relu,height=13.928,width=4.8773",5,4.877261,13.928334,relu
daef1_00003,4,9.571207,0.110941,True,None,None,5,-9.571207,539233b11eaf47d2bc9702ffbafd8250,2022-03-16_16-00-23,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.535944,0,5,"3_activation=tanh,height=71.118,width=7.6651",5,7.665065,71.118037,tanh
daef1_00004,4,1.193528,0.106211,True,None,None,5,-1.193528,b113026e09fb42d9bd3b1988376bf115,2022-03-16_16-00-23,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.526737,0,5,"4_activation=relu,height=-4.9967,width=12.265",5,12.264920,-4.996748,relu


In [8]:
analysis.trials

[easy_objective_daef1_00000,
 easy_objective_daef1_00001,
 easy_objective_daef1_00002,
 easy_objective_daef1_00003,
 easy_objective_daef1_00004,
 easy_objective_daef1_00005,
 easy_objective_daef1_00006,
 easy_objective_daef1_00007,
 easy_objective_daef1_00008,
 easy_objective_daef1_00009]

In [9]:
analysis.dataframe(metric="mean_loss", mode="min").head(5)

,iterations,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,trial_id,experiment_id,...,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/activation,config/height,config/steps,config/width,logdir
0,4,10.403145,0.102530,False,NaN,NaN,5,-10.403145,daef1_00000,f045f83e5eb94e50bb47b023f4e1ac90,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.528258,0,5,relu,84.480597,5,10.287164,/Users/jules/ray_results/easy_objective_2022-0...
1,4,6.399146,0.128668,False,NaN,NaN,5,-6.399146,daef1_00001,6589dda1beb94cd59fcb3baaf0ad6fa6,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.551824,0,5,tanh,32.544925,5,5.450001,/Users/jules/ray_results/easy_objective_2022-0...
2,4,4.781625,0.105659,False,NaN,NaN,5,-4.781625,daef1_00002,5b214c0fc1854efe9300e12eef47e3e0,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.520012,0,5,relu,13.928334,5,4.877261,/Users/jules/ray_results/easy_objective_2022-0...
3,4,9.571207,0.110941,False,NaN,NaN,5,-9.571207,daef1_00003,539233b11eaf47d2bc9702ffbafd8250,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.535944,0,5,tanh,71.118037,5,7.665065,/Users/jules/ray_results/easy_objective_2022-0...
4,4,1.193528,0.106211,False,NaN,NaN,5,-1.193528,daef1_00004,b113026e09fb42d9bd3b1988376bf115,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.526737,0,5,relu,-4.996748,5,12.264920,/Users/jules/ray_results/easy_objective_2022-0...


In [10]:
ray.shutdown()